In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
params = setups["coveo"]["params_bprmax"]

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_gru4rec_pytorch_script(model_name, train_folder, train_data, test_data, model_path, loss, optim, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, n_epochs, m, eval_hidden_reset, use_correct_loss, use_correct_mask_reset):
    checkpoint_dir = f"{model_path}/{model_name}"
    s_train_full = f"CUDA_VISIBLE_DEVICES=0 python ./GRU4REC-pytorch/main.py --data_folder {train_folder} --train_data {train_data} --valid_data {test_data} --checkpoint_dir {checkpoint_dir} --num_layers {1} --embedding_dim {layers} --hidden_size {layers} --loss_type {'BPR-max' if loss=='bpr-max' else 'CrossEntropy'} --final_act {final_act} --n_epochs {n_epochs} --batch_size {batch_size} --dropout_input {dropout_p_embed} --dropout_hidden {dropout_p_hidden} --lr {learning_rate} --momentum {0.0} --optimizer_type {'Adagrad' if optim=='adagrad' else ''}{' --eval_hidden_reset' if eval_hidden_reset else ''}{' --use_correct_loss' if use_correct_loss else ''}{' --use_correct_mask_reset' if use_correct_mask_reset else ''}"
    s_test_full = s_train_full + f" --is_eval --load_model {checkpoint_dir}/model_0000{n_epochs-1}.pt --m {m}"
    return s_train_full, s_test_full

In [ ]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [ ]:
train_folder, train_data = '/'.join(train_path.split('/')[:-1]), train_path.split('/')[-1]
test_folder, test_data = '/'.join(test_path.split('/')[:-1]), test_path.split('/')[-1]

In [ ]:
train_script_oob, test_script_oob = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_oob_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=False, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box model

In [ ]:
os.system(train_script_oob)

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)

## Train & test inference fix model

In [ ]:
train_script_inffix, test_script_inffix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_inffix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box eval fix model

In [ ]:
os.system(train_script_inffix)

### Test the out-of-the-box eval fix model

In [ ]:
os.system(test_script_inffix)

## Train & test the major fix model

In [ ]:
train_script_majorfix, test_script_majorfix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_majorfix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=True, use_correct_mask_reset=True)

### Train the major fix model

In [ ]:
os.system(train_script_majorfix)

### Test the major fix model

In [ ]:
os.system(test_script_majorfix)